In [1]:
%load_ext autoreload
%autoreload 2

import duckdb
import json
import os
import pandas as pd
import pathlib

# from ibge import *
from utils import *

In [2]:
# Initialize DuckDB connection
conn = duckdb.connect("Piper.db")
!ls gbif/

gbif_classifications.parquet  gbif.parquet  Piper.js+json.zip


## GBIF
### Adiciono os registros no duckdb (via parquet)

In [ ]:
local_file = "./gbif/gbif.parquet"

conn.execute("DROP TABLE IF EXISTS gbif")
conn.execute(f"""
    CREATE TABLE gbif AS 
    SELECT * FROM read_parquet('{local_file}')
""")

result = conn.execute("SELECT COUNT(*) FROM gbif").fetchone()
print(f"Registros carregados na tabela: {result[0]:,}")

Registros carregados na tabela: 27,556


In [7]:
conn.execute("DESCRIBE gbif").df()

,column_name,column_type,null,key,default,extra
0,key,BIGINT,YES,None,None,None
1,datasetKey,VARCHAR,YES,None,None,None
2,publishingOrgKey,VARCHAR,YES,None,None,None
3,networkKeys,VARCHAR[],YES,None,None,None
4,installationKey,VARCHAR,YES,None,None,None
...,...,...,...,...,...,...
239,depth,DOUBLE,YES,None,None,None
240,depthAccuracy,DOUBLE,YES,None,None,None
241,waterBody,VARCHAR,YES,None,None,None
242,associatedSequences,VARCHAR,YES,None,None,None


In [8]:
conn.execute("SELECT * FROM gbif LIMIT 5").df()

,key,datasetKey,publishingOrgKey,networkKeys,installationKey,hostingOrganizationKey,publishingCountry,protocol,lastCrawled,lastParsed,...,extensions.http://data.ggbn.org/schemas/ggbn/terms/Preparation,parentNameUsageID,originalNameUsage,footprintSRS,higherGeographyID,depth,depthAccuracy,waterBody,associatedSequences,island
0,2998027570,a185d0c4-1a12-45c1-a729-2a7b058b528d,838537fc-4aeb-4980-8b58-97fbdcd1d91a,[4b0d8edb-7504-42c4-9349-63e86c01bf97],ecb15bdf-98de-4937-84f5-4f93ee9efc73,838537fc-4aeb-4980-8b58-97fbdcd1d91a,BR,EML,2025-11-01T12:14:04.613+00:00,2025-11-04T22:54:47.632+00:00,...,<NA>,None,None,None,None,NaN,NaN,None,None,None
1,3390903613,26d97e94-6ee9-4d5a-a9b8-7d514ec0345c,073e52d4-44bd-41d7-bdfa-88c2735c694b,<NA>,68845aab-a453-4816-b36d-a3ec8455deb0,c4f66525-4d36-4c18-82ac-98e088f54db4,CO,EML,2025-11-04T05:25:28.096+00:00,2025-11-05T02:58:47.070+00:00,...,<NA>,None,None,None,None,NaN,NaN,None,None,None
2,4031562717,7bd65a7a-f762-11e1-a439-00145eb45e9a,90fd6680-349f-11d8-aa2d-b8a03c50a862,<NA>,c4e134d3-c68f-406d-a116-6a37895602e0,90fd6680-349f-11d8-aa2d-b8a03c50a862,US,EML,2025-11-19T04:22:10.786+00:00,2025-11-19T05:10:39.243+00:00,...,<NA>,None,None,None,None,NaN,NaN,None,None,None
3,4031950051,7bd65a7a-f762-11e1-a439-00145eb45e9a,90fd6680-349f-11d8-aa2d-b8a03c50a862,<NA>,c4e134d3-c68f-406d-a116-6a37895602e0,90fd6680-349f-11d8-aa2d-b8a03c50a862,US,EML,2025-11-19T04:22:10.786+00:00,2025-11-19T05:11:05.005+00:00,...,<NA>,None,None,None,None,NaN,NaN,None,None,None
4,4061265601,e053ff53-c156-4e2e-b9b5-4462e9625424,90fd6680-349f-11d8-aa2d-b8a03c50a862,<NA>,c4e134d3-c68f-406d-a116-6a37895602e0,90fd6680-349f-11d8-aa2d-b8a03c50a862,US,EML,2025-11-19T04:07:10.017+00:00,2025-11-19T04:35:27.921+00:00,...,<NA>,None,None,None,None,NaN,NaN,None,None,None


### GBIF
#### Lista de sinônimos

In [3]:
local_file = "./gbif/gbif_classifications.parquet"

conn.execute("DROP TABLE IF EXISTS gbif_classifications")
conn.execute(f"""
    CREATE TABLE gbif_classifications AS 
    SELECT DISTINCT(*) FROM read_parquet('{local_file}')
""")

result = conn.execute("SELECT COUNT(*) FROM gbif_classifications").fetchone()
print(f"Registros carregados na tabela: {result[0]:,}")

Registros carregados na tabela: 128


In [4]:
conn.execute("DESCRIBE gbif_classifications").df()

,column_name,column_type,null,key,default,extra
0,species,VARCHAR,YES,None,None,None
1,taxonomicStatus,VARCHAR,YES,None,None,None


In [5]:
conn.execute("SELECT * FROM gbif_classifications LIMIT 5").df()

,species,taxonomicStatus
0,Piper imperiale,ACCEPTED
1,Piper brachipilum,ACCEPTED
2,Piper begoniicolor,ACCEPTED
3,Piper curtispicum,PROVISIONALLY_ACCEPTED
4,Piper schiedeanum,ACCEPTED


### IUCN

In [25]:
local_file = "./iucn/red_list_iucn.parquet"
conn.execute("DROP TABLE IF EXISTS iucn")
conn.execute(f"""
        CREATE TABLE iucn AS
        SELECT * FROM read_parquet('{local_file}', union_by_name=True)
    """)

result = conn.execute("SELECT COUNT(*) FROM iucn").fetchone()
print(f"Registros carregados na tabela: {result[0]:,}")

Registros carregados na tabela: 122


In [26]:
conn.execute("DESCRIBE iucn").df()

,column_name,column_type,null,key,default,extra
0,internalTaxonId,BIGINT,YES,None,None,None
1,scientificName_x,VARCHAR,YES,None,None,None
2,kingdomName,VARCHAR,YES,None,None,None
3,phylumName,VARCHAR,YES,None,None,None
4,className,VARCHAR,YES,None,None,None
5,orderName,VARCHAR,YES,None,None,None
6,familyName,VARCHAR,YES,None,None,None
7,genusName,VARCHAR,YES,None,None,None
8,speciesName,VARCHAR,YES,None,None,None
9,infraType,DOUBLE,YES,None,None,None


In [27]:
conn.execute("SELECT * FROM iucn LIMIT 5").df()

,internalTaxonId,scientificName_x,kingdomName,phylumName,className,orderName,familyName,genusName,speciesName,infraType,...,populationTrend,range,useTrade,systems,conservationActions,realm,yearLastSeen,possiblyExtinct,possiblyExtinctInTheWild,scopes
0,45862,Piper skutchii,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,skutchii,NaN,...,None,None,None,Terrestrial,None,Neotropical,NaN,False,False,Global
1,45861,Piper schuppii,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,schuppii,NaN,...,None,None,None,Terrestrial,None,Neotropical,NaN,False,False,Global
2,45863,Piper sodiroi,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,sodiroi,NaN,...,None,None,None,Terrestrial,None,Neotropical,NaN,False,False,Global
3,45864,Piper stipulosum,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,stipulosum,NaN,...,None,None,None,Terrestrial,None,Neotropical,NaN,False,False,Global
4,45814,Piper achupallasense,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,achupallasense,NaN,...,None,Endemic to shrubby paramo in southern Ecuador....,None,Terrestrial,Not confirmed in any Ecuadorean protected area.,Neotropical,NaN,False,False,Global


### POWO

In [3]:
local_file = "./powo/powo.parquet"
conn.execute("DROP TABLE IF EXISTS powo")
conn.execute(f"""
        CREATE TABLE powo AS
        SELECT * FROM read_parquet('{local_file}', union_by_name=True)
    """)

result = conn.execute("SELECT COUNT(*) FROM powo").fetchone()
print(f"Registros carregados na tabela: {result[0]:,}")

Registros carregados na tabela: 156


In [4]:
conn.execute("SELECT * FROM powo LIMIT 5").df()

,species,name,url,publicacao,native_countries,native_countries_count,iucn,iucn_code,iucn_full_name,status_species,native_range
0,Piper san-miguelense,None,https://powo.science.kew.org/taxon/urn:lsid:ip...,First published in Piperac. N. South Amer.: 28...,Ecuador,1,DD - data,DD,data,This species is accepted,The native range of this species is Ecuador. I...
1,Piper schuppii,None,https://powo.science.kew.org/taxon/urn:lsid:ip...,First published in Phytologia 47: 106 (1980),Ecuador,1,VU - vulnerable,VU,vulnerable,This species is accepted,The native range of this species is Ecuador. I...
2,Piper sodiroi,None,https://powo.science.kew.org/taxon/urn:lsid:ip...,First published in Bull. Herb. Boissier 6: 482...,Ecuador,1,VU - vulnerable,VU,vulnerable,This species is accepted,The native range of this species is Ecuador. I...
3,Piper stipulosum,None,https://powo.science.kew.org/taxon/urn:lsid:ip...,First published in Piperac. Ecuator.: 148 (1900),Ecuador,1,CR - critically,CR,critically,This species is accepted,The native range of this species is Ecuador. I...
4,Piper achupallasense,None,https://powo.science.kew.org/taxon/urn:lsid:ip...,"First published in W.Trelease & T.G.Yuncker, P...",Ecuador,1,EN - endangered,EN,endangered,This species is accepted,The native range of this species is Ecuador. I...


In [6]:
conn.close()